### Baseline Experiment: 
- Prepare the Query Embedding
- Retrieve Abstracts and Compute Embeddings
- Calculate Cosine Similarity based on semantic_search
- Select the Best Abstract for Each Edge
- Ensure at Least One Selection per Edge

In [3]:
import torch
import json
import time
import os
import pandas as pd
import numpy as np
from pathlib import Path
from tqdm import tqdm
import random
import text_util
import openai
from transformers import DPRQuestionEncoder, DPRQuestionEncoderTokenizer, DPRContextEncoder, DPRContextEncoderTokenizer
from openai import OpenAI
from agatha.construct.semrep_handler import SemRepHandler
from agatha.util.sqlite3_lookup import Sqlite3LookupTable
from sentence_transformers import SentenceTransformer, util
import warnings
from openai_llm import oai_get_response
warnings.filterwarnings( ignore )
os.environ[ 'TOKENIZERS_PARALLELISM '] =  'false '

In [2]:
agatha_sent_db_path = (
     /lustre/acslab/shared/Agatha_shared/2021_11_22_full/sentences.sqlite3 
)
sents_db = Sqlite3LookupTable(agatha_sent_db_path)

In [3]:
pd.set_option( display.max_columns , None)
pd.set_option( display.max_colwidth , None)

In [4]:
df = pd.read_pickle( P3_SLC6A3_Bupropion.pkl )
top_20_df = df.loc[:].head(20)
top_20_df

,path,score_std,score_mean,dec_path,context_pmids
220,"[C1420213, C0013124, C0424166, C0085208]",2.463338e-06,0.999911,"[SLC6A3 gene, Drinking behavior processes, Social Anxiety, bupropion]","{('C1420213', 'C0013124'): ['19352220'], ('C0013124', 'C0424166'): ['30138776', '29175292', '24841178', '26235528'], ('C0424166', 'C0085208'): ['28591669']}"
339,"[C1420213, C1417965, C0030662, C0085208]",6.882552e-08,0.999898,"[SLC6A3 gene, OPRM1 gene, Gambling, Pathological, bupropion]","{('C1420213', 'C1417965'): ['9790747', '22551036'], ('C1417965', 'C0030662'): ['31553235'], ('C0030662', 'C0085208'): ['14709965', '29692743', '16282845', '17414236']}"
114,"[C1420213, C1417965, C0001969, C0085208]",0.000000e+00,0.999884,"[SLC6A3 gene, OPRM1 gene, Alcoholic Intoxication, bupropion]","{('C1420213', 'C1417965'): ['9790747', '22551036'], ('C1417965', 'C0001969'): ['21039637', '17207095', '20528823'], ('C0001969', 'C0085208'): ['1452106']}"
195,"[C1420213, C1439340, C3539788, C0085208]",2.038944e-05,0.999809,"[SLC6A3 gene, SLC6A3 protein, human, Synaptic Vesicular Amine Transporter, bupropion]","{('C1420213', 'C1439340'): ['17171650', '32102440', '17565601', '25684044', '26364979'], ('C1439340', 'C3539788'): ['32991141', '20830722'], ('C3539788', 'C0085208'): ['16923164', '15322260']}"
180,"[C1420213, C1439340, C0020651, C0085208]",5.052018e-05,0.999795,"[SLC6A3 gene, SLC6A3 protein, human, Hypotension, Orthostatic, bupropion]","{('C1420213', 'C1439340'): ['17171650', '32102440', '17565601', '25684044', '26364979'], ('C1439340', 'C0020651'): ['35508570'], ('C0020651', 'C0085208'): ['6406451', '6803273']}"
141,"[C1420213, C0013124, C0023896, C0085208]",0.000000e+00,0.999781,"[SLC6A3 gene, Drinking behavior processes, Alcoholic Liver Diseases, bupropion]","{('C1420213', 'C0013124'): ['19352220'], ('C0013124', 'C0023896'): ['14634491', '11410736', '9194910'], ('C0023896', 'C0085208'): ['2124217']}"
102,"[C1420213, C1417965, C0221745, C0085208]",0.000000e+00,0.999772,"[SLC6A3 gene, OPRM1 gene, Depression suicidal, bupropion]","{('C1420213', 'C1417965'): ['9790747', '22551036'], ('C1417965', 'C0221745'): ['32590406', '34703230'], ('C0221745', 'C0085208'): ['24107760']}"
309,"[C1420213, C1760428, C0085208]",0.000000e+00,0.999764,"[SLC6A3 gene, Suicidal behavior, bupropion]","{('C1420213', 'C1760428'): ['34199792'], ('C1760428', 'C0085208'): ['21993207']}"
203,"[C1420213, C1417965, C0270549, C0085208]",0.000000e+00,0.999724,"[SLC6A3 gene, OPRM1 gene, Generalized Anxiety Disorder, bupropion]","{('C1420213', 'C1417965'): ['9790747', '22551036'], ('C1417965', 'C0270549'): ['23658070'], ('C0270549', 'C0085208'): ['18362870', '1541607']}"
205,"[C1420213, C0001957, C0527316, C0085208]",0.000000e+00,0.999707,"[SLC6A3 gene, Alcohol Withdrawal Delirium, donepezil, bupropion]","{('C1420213', 'C0001957'): ['12513948'], ('C0001957', 'C0527316'): ['14629709'], ('C0527316', 'C0085208'): ['34412107']}"


In [5]:
model_sbert = SentenceTransformer( all-MiniLM-L6-v2 )  

# Data structure containing PMIDs for each edge
# edges_pmids = {
#     ( C1420213 ,  C0013124 ): [ 19352220 ],
#     ( C0013124 ,  C0424166 ): [ 30138776 ,  29175292 ,  24841178 ,  26235528 ],
#     ( C0424166 ,  C0085208 ): [ 28591669 ]
# }

In [6]:
def get_embedding(text):
    return model_sbert.encode(text, convert_to_tensor=True)

def retrieve_single_abstracts_from_pmid(pmid):
    abstr_text = text_util.get_abstr_text(pmid, sents_db)
    return abstr_text

def retrieve_abstracts_from_pmids(pmids, sents_db):
    abstracts = []
    
    for pmid in tqdm(pmids, desc= 'Retrieving abstracts '):
        abstr_text = text_util.get_abstr_text(pmid, sents_db)
        abstracts.append(abstr_text)
    
    return abstracts

def find_best_selection_context_per_path_sbert(source, target, edges_pmids):
    #complexity of query pairwise can improve?
    query = f '{source} {target} '
    query_embedding = get_embedding(query)
    
    best_selection = {}
    cosine_similarities = {}
    for edge, pmids in edges_pmids.items():
        pmid_texts = [retrieve_single_abstracts_from_pmid(pmid) for pmid in pmids]
        pmid_embeddings = model_sbert.encode(pmid_texts, convert_to_tensor=True, show_progress_bar=False)
        
        # Perform semantic search which use cos_sim by default according to sourcecode https://github.com/UKPLab/sentence-transformers/blob/master/sentence_transformers/util.py#L124
        search_results = util.semantic_search(query_embedding, pmid_embeddings, top_k=len(pmid_embeddings))
        
        # the best match and its cos_sim
        best_pmid_index = search_results[0][0][ corpus_id ]
        best_pmid = pmids[best_pmid_index]
        best_score = search_results[0][0][ score ]
        
        best_selection[edge] = best_pmid
        cosine_similarities[edge] = best_score
    
    return best_selection, cosine_similarities

In [7]:
def process_dataframe_row(row):
    #make sure you use umls term to do embedding source and target
    source =  'C1420213 ' #   'SLC6A3 '
    target =  'C0085208 ' #   'Bupropion '
    edges_pmids = row[ context_pmids ]
    
    best_selection, cosine_similarities = find_best_selection_context_per_path_sbert(source, target, edges_pmids)
    return pd.Series({ best_selection_sbert : best_selection,  cosine_similarity : cosine_similarities})

In [8]:
def generate_llm_response(row):
    best_selection = row[ best_selection_sbert ]
    pmids = list(best_selection.values())
    print( Here is the list of PMIDs according to sbert: ,  ,  .join(map(str, pmids)))
    
    # Retrieve context abstracts from HGCR
    path_context_abstracts = retrieve_abstracts_from_pmids(pmids, sents_db)
    
    # prepare  prompt
    source_cui =  SLC6A3 
    target_cui =  Bupropion 
    llm_prompt_template =  How would you describe an indirect relationship between {source} and {target} given the following scientific abstracts as contexts? 
    llm_fix_prompt_str =  \n\n .join(path_context_abstracts)
    llm_fix_prompt_combo = llm_prompt_template.format(source=source_cui, target=target_cui) +  \n\n  + llm_fix_prompt_str
    
    # measure time for generating response from gpt4o
    start_time = time.time()
    llm_resp_path = oai_get_response(llm_fix_prompt_combo, temp=1e-19, top_p=1e-9, seed=1234)
    end_time = time.time()
    
    duration = end_time - start_time
    print(f 'Time taken for oai_get_response: {duration:.2f} seconds ')
    
    return llm_resp_path

### Result when I choose only one pmid per bucket/edge in each path:

In [9]:
intermediate_df = top_20_df.apply(process_dataframe_row, axis=1)

res_df = top_20_df.copy()
res_df[[ best_selection_sbert ,  cosine_similarity ]] = intermediate_df
res_df[ llm_response_No_Feedback_Baseline ] = res_df.apply(generate_llm_response, axis=1)

res_df

Here is the list of PMIDs according to sbert: 19352220, 26235528, 28591669


Retrieving abstracts: 100%|██████████| 3/3 [00:00<00:00, 21546.08it/s]


Time taken for oai_get_response: 7.80 seconds
Here is the list of PMIDs according to sbert: 22551036, 31553235, 14709965


Retrieving abstracts: 100%|██████████| 3/3 [00:00<00:00, 18780.47it/s]


Time taken for oai_get_response: 5.07 seconds
Here is the list of PMIDs according to sbert: 22551036, 21039637, 1452106


...

Retrieving abstracts: 100%|██████████| 3/3 [00:00<00:00, 11428.62it/s]


Time taken for oai_get_response: 8.29 seconds
Here is the list of PMIDs according to sbert: 22551036, 20668445, 33806634


Retrieving abstracts: 100%|██████████| 3/3 [00:00<00:00, 10681.59it/s]


Time taken for oai_get_response: 6.80 seconds


,path,score_std,score_mean,dec_path,context_pmids,best_selection_sbert,cosine_similarity,llm_response_No_Feedback_Baseline
220,"[C1420213, C0013124, C0424166, C0085208]",2.463338e-06,0.999911,"[SLC6A3 gene, Drinking behavior processes, Social Anxiety, bupropion]","{('C1420213', 'C0013124'): ['19352220'], ('C0013124', 'C0424166'): ['30138776', '29175292', '24841178', '26235528'], ('C0424166', 'C0085208'): ['28591669']}","{('C1420213', 'C0013124'): '19352220', ('C0013124', 'C0424166'): '26235528', ('C0424166', 'C0085208'): '28591669'}","{('C1420213', 'C0013124'): 0.08569406718015671, ('C0013124', 'C0424166'): 0.0020800335332751274, ('C0424166', 'C0085208'): 0.09995880722999573}","An indirect relationship between SLC6A3 (the gene encoding the dopamine transporter, DAT1) and Bupropion can be inferred from the provided scientific abstracts through their respective roles in dopaminergic neurotransmission and behavioral outcomes related to alcohol use and social anxiety.\n\n1. **SLC6A3 and Alcohol Consumption**:\n - The first abstract discusses the association between genetic variations in SLC6A3 and alcohol consumption behavior in a Finnish male population. It highlights that certain polymorphisms in the DAT1 gene are linked to alcohol consumption and dependence, suggesting that DAT1 plays a significant role in regulating dopaminergic neurotransmission, which in turn influences drinking behavior.\n\n2. **Social Anxiety and Alcohol Use**:\n - The second abstract explores the relationship between social anxiety, disengagement coping, and alcohol use among adolescents. It suggests that socially anxious adolescents may use alcohol as a coping mechanism to manage negative affectivity, indicating a behavioral link between social anxiety and alcohol consumption.\n\n3. **Bupropion and Social Anxiety**:\n - The third abstract examines the effects of Bupropion on social behavior in adolescent mice, noting that the drug induces social anxiety-like behaviors, particularly under certain housing conditions. Bupropion is known to affect dopaminergic and noradrenergic systems, which are also implicated in the regulation of mood and anxiety.\n\n### Indirect Relationship:\n\n- **Dopaminergic System**: Both SLC6A3 and Bupropion are involved in the dopaminergic system. SLC6A3 encodes the dopamine transporter, which is crucial for dopamine reuptake and regulation of dopaminergic signaling. Bupropion, on the other hand, inhibits the reuptake of dopamine and norepinephrine, thereby increasing their levels in the synaptic cleft.\n\n- **Behavioral Outcomes**: Variations in SLC6A3 are associated with alcohol consumption behaviors, potentially due to their impact on dopamine regulation. Bupropion, by modulating dopamine levels, can influence behaviors related to social anxiety and potentially alcohol use, given its anxiogenic effects observed in the study on mice.\n\n- **Alcohol Use and Social Anxiety**: The second abstract links social anxiety with alcohol use as a coping mechanism. Since Bupropion can induce social anxiety-like behaviors, it might indirectly affect alcohol consumption behaviors in individuals with social anxiety, similar to the genetic variations in SLC6A3.\n\n### Conclusion:\n\nThe indirect relationship between SLC6A3 and Bupropion can be described as follows: SLC6A3 genetic variations influence dopaminergic neurotransmission, which affects alcohol consumption behaviors. Bupropion, by altering dopamine levels, can induce social anxiety-like behaviors, which may lead to increased alcohol use as a coping mechanism in socially anxious individuals. Thus, both SLC6A3 and Bupropion impact dopaminergic pathways that are crucial for understanding behaviors related to alcohol use and social anxiety."
339,"[C1420213, C1417965, C0030662, C0085208]",6.882552e-08,0.999898,"[SLC6A3 gene, OPRM1 gene, Gambling, Pathological, bupropion]","{('C1420213', 'C1417965'): ['9790747', '22551036'], ('C1417965', 'C0030662'): ['31553235'], ('C0030662', 'C0085208'): [

In [60]:



res_df.to_pickle( Baseline_SBERT.pkl )

In [15]:
df_final = pd.read_pickle( 'Baseline_SBERT.pkl' )

In [19]:
with open('fdbloop_result_p3.txt', 'r') as file:
    content = file.read()
responses = content.split('---')
df_final['llm_response_feedback_loop'] = [response.strip() for response in responses]
df_final

,path,score_std,score_mean,dec_path,context_pmids,best_selection_sbert,cosine_similarity,llm_response_No_Feedback_Baseline,llm_response_feedback_loop
220,"[C1420213, C0013124, C0424166, C0085208]",2.463338e-06,0.999911,"[SLC6A3 gene, Drinking behavior processes, Soc...","{('C1420213', 'C0013124'): ['19352220'], ('C00...","{('C1420213', 'C0013124'): '19352220', ('C0013...","{('C1420213', 'C0013124'): 0.08569406718015671...",An indirect relationship between SLC6A3 (the g...,"Given the provided scientific abstracts, an in..."
339,"[C1420213, C1417965, C0030662, C0085208]",6.882552e-08,0.999898,"[SLC6A3 gene, OPRM1 gene, Gambling, Pathologic...","{('C1420213', 'C1417965'): ['9790747', '225510...","{('C1420213', 'C1417965'): '22551036', ('C1417...","{('C1420213', 'C1417965'): 0.10667748004198074...","Based on the provided abstracts, an indirect r...","Given the provided scientific abstracts, an in..."
114,"[C1420213, C1417965, C0001969, C0085208]",0.000000e+00,0.999884,"[SLC6A3 gene, OPRM1 gene, Alcoholic Intoxicati...","{('C1420213', 'C1417965'): ['9790747', '225510...","{('C1420213', 'C1417965'): '22551036', ('C1417...","{('C1420213', 'C1417965'): 0.10667748004198074...","Based on the provided abstracts, an indirect r...","Given the provided scientific abstracts, an in..."
195,"[C1420213, C1439340, C3539788, C0085208]",2.038944e-05,0.999809,"[SLC6A3 gene, SLC6A3 protein, human, Synaptic ...","{('C1420213', 'C1439340'): ['17171650', '32102...","{('C1420213', 'C1439340'): '17565601', ('C1439...","{('C1420213', 'C1439340'): 0.1357736587524414,...",An indirect relationship between SLC6A3 and Bu...,An indirect relationship between SLC6A3 (the g...
180,"[C1420213, C1439340, C0020651, C0085208]",5.052018e-05,0.999795,"[SLC6A3 gene, SLC6A3 protein, human, Hypotensi...","{('C1420213', 'C1439340'): ['17171650', '32102...","{('C1420213', 'C1439340'): '17565601', ('C1439...","{('C1420213', 'C1439340'): 0.1357736587524414,...",An indirect relationship between SLC6A3 (the g...,"Given the provided abstracts, an indirect rela..."
141,"[C1420213, C0013124, C0023896, C0085208]",0.000000e+00,0.999781,"[SLC6A3 gene, Drinking behavior processes, Alc...","{('C1420213', 'C0013124'): ['19352220'], ('C00...","{('C1420213', 'C0013124'): '19352220', ('C0013...","{('C1420213', 'C0013124'): 0.08569406718015671...",An indirect relationship between SLC6A3 (the g...,An indirect relationship between SLC6A3 (the g...
102,"[C1420213, C1417965, C0221745, C0085208]",0.000000e+00,0.999772,"[SLC6A3 gene, OPRM1 gene, Depression suicidal,...","{('C1420213', 'C1417965'): ['9790747', '225510...","{('C1420213', 'C1417965'): '22551036', ('C1417...","{('C1420213', 'C1417965'): 0.10667748004198074...",An indirect relationship between SLC6A3 (the g...,"Given the provided scientific abstracts, an in..."
309,"[C1420213, C1760428, C0085208]",0.000000e+00,0.999764,"[SLC6A3 gene, Suicidal behavior, bupropion]","{('C1420213', 'C1760428'): ['34199792'], ('C17...","{('C1420213', 'C1760428'): '34199792', ('C1760...","{('C1420213', 'C1760428'): 0.09907166659832001...",An indirect relationship between SLC6A3 (DAT1)...,Given the context provided by the scientific a...
203,"[C1420213, C1417965, C0270549, C0085208]",0.000000e+00,0.999724,"[SLC6A3 gene, OPRM1 gene, Generalized Anxiety ...","{('C1420213', 'C1417965'): ['9790747', '225510...","{('C1420213', 'C1417965'): '22551036', ('C1417...","{('C1420213', 'C1417965'): 0.10667748004198074...","Based on the provided abstracts, an indirect r...","Given the provided scientific abstracts, an in..."
205,"[C1420213, C0001957, C0527316, C0085208]",0.000000e+00,0.999707,"[SLC6A3 gene, Alcohol Withdrawal Delirium, don...","{('C1420213', 'C0001957'): ['12513948'], ('C00...","{('C1420213', 'C0001957'): '12513948', ('C0001...","{('C1420213', 'C0001957'): 0.04149764403700828...","Based on the provided abstracts, an indirect r...","Based on the provided abstracts, an indirect r..."


In [21]:
df_final.to_pickle('df_final.pkl' )

In [20]:
df_final.to_csv('df_final.csv', index=False)

print("DataFrame has been written to updated_dataframe.csv")

DataFrame has been written to updated_dataframe.csv
